# Libraries & dependencies

In [ ]:
!pip install pystac
!pip install pystac_client
!pip install planetary_computer
!pip install nb_black
!pip install odc-stac
!pip install geopandas
!pip install rioxarray
!pip install loguru
!pip install opencv-python
!pip install geopy
!pip install path
!pip install tqdm

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import odc.stac
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

%matplotlib inline

In [ ]:
DATA_DIR = Path.cwd().parent.resolve() / r"Documents\ticktickbloom"
assert DATA_DIR.exists()

# **metadata.csv**

In [ ]:
metadata = pd.read_csv(DATA_DIR / "metadata.csv")
metadata.head()

In [ ]:
metadata.info()

# **train_labels.csv**

In [ ]:
train_labels = pd.read_csv(DATA_DIR / "train_labels.csv")
train_labels.head()

In [ ]:
train_labels.info()

In [ ]:
train_labels["severity"].hist()

In [ ]:
train_labels["density"].describe()

In [ ]:
import seaborn as sns

sns.distplot(
    np.log(
        train_labels[(train_labels["density"] > 0) & (train_labels["density"] < 1e7)][
            "density"
        ]
    )
)

In [ ]:
sns.heatmap(train_labels.corr(), annot=True, cmap="coolwarm", center=0)

DATA_merge

In [ ]:
data = train_labels.merge(
    metadata, how="left", left_on="uid", right_on="uid", validate="1:1"
)
data

In [ ]:
sns.heatmap(data.corr(), annot=True, cmap="coolwarm", center=0)

In [ ]:
data["region"].hist()

In [ ]:
data[data["region"] == "midwest"]["severity"].hist()

In [ ]:
data[data["region"] == "south"]["severity"].hist()

In [ ]:
data[data["region"] == "west"]["severity"].hist()

In [ ]:
data[data["region"] == "northeast"]["severity"].hist()

# **Process feature data**

In [ ]:
import rioxarray
from IPython.display import Image
from PIL import Image as PILImage

In [ ]:
# Establish a connection to the STAC API
import planetary_computer as pc
from pystac_client import Client

catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace
)

In [ ]:
import geopy.distance as distance


# get our bounding box to search latitude and longitude coordinates
def get_bounding_box(latitude, longitude, meter_buffer=50000):
    """
    Given a latitude, longitude, and buffer in meters, returns a bounding
    box around the point with the buffer on the left, right, top, and bottom.

    Returns a list of [minx, miny, maxx, maxy]
    """
    distance_search = distance.distance(meters=meter_buffer)

    # calculate the lat/long bounds based on ground distance
    # bearings are cardinal directions to move (south, west, north, and east)
    min_lat = distance_search.destination((latitude, longitude), bearing=180)[0]
    min_long = distance_search.destination((latitude, longitude), bearing=270)[1]
    max_lat = distance_search.destination((latitude, longitude), bearing=0)[0]
    max_long = distance_search.destination((latitude, longitude), bearing=90)[1]

    return [min_long, min_lat, max_long, max_lat]


# get our date range to search, and format correctly for query
def get_date_range(date, time_buffer_days=15):
    """Get a date range to search for in the planetary computer based
    on a sample's date. The time range will include the sample date
    and time_buffer_days days prior

    Returns a string"""
    datetime_format = "%Y-%m-%dT"
    range_start = pd.to_datetime(date) - timedelta(days=time_buffer_days)
    date_range = f"{range_start.strftime(datetime_format)}/{pd.to_datetime(date).strftime(datetime_format)}"

    return date_range


def crop_sentinel_image(item, bounding_box):
    """
    Given a STAC item from Sentinel-2 and a bounding box tuple in the format
    (minx, miny, maxx, maxy), return a cropped portion of the item's visual
    imagery in the bounding box.

    Returns the image as a numpy array with dimensions (color band, height, width)
    """
    (minx, miny, maxx, maxy) = bounding_box

    image = rioxarray.open_rasterio(pc.sign(item.assets["visual"].href)).rio.clip_box(
        minx=minx,
        miny=miny,
        maxx=maxx,
        maxy=maxy,
        crs="EPSG:4326",
    )

    return image.to_numpy()


def crop_landsat_image(item, bounding_box):
    """
    Given a STAC item from Landsat and a bounding box tuple in the format
    (minx, miny, maxx, maxy), return a cropped portion of the item's visual
    imagery in the bounding box.

    Returns the image as a numpy array with dimensions (color band, height, width)
    """
    (minx, miny, maxx, maxy) = bounding_box

    image = odc.stac.stac_load(
        [pc.sign(item)], bands=["red", "green", "blue"], bbox=[minx, miny, maxx, maxy]
    ).isel(time=0)
    image_array = image[["red", "green", "blue"]].to_array().to_numpy()

    # normalize to 0 - 255 values
    image_array = cv2.normalize(image_array, None, 0, 255, cv2.NORM_MINMAX)

    return image_array


# Refactor our process from above into functions
# Refactor our process from above into functions
def select_best_item(items, date, latitude, longitude):
    """
    Select the best satellite item given a sample's date, latitude, and longitude.
    If any Sentinel-2 imagery is available, returns the closest sentinel-2 image by
    time. Otherwise, returns the closest Landsat imagery.

    Returns a tuple of (STAC item, item platform name, item date)
    """
    # get item details
    item_details = pd.DataFrame(
        [
            {
                "datetime": item.datetime.strftime("%Y-%m-%d"),
                "platform": item.properties["platform"],
                "min_long": item.bbox[0],
                "max_long": item.bbox[2],
                "min_lat": item.bbox[1],
                "max_lat": item.bbox[3],
                "item_obj": item,
            }
            for item in items
        ]
    )

    # filter to items that contain the point location, or return None if none contain the point
    item_details["contains_sample_point"] = (
        (item_details.min_lat < latitude)
        & (item_details.max_lat > latitude)
        & (item_details.min_long < longitude)
        & (item_details.max_long > longitude)
    )
    item_details = item_details[item_details["contains_sample_point"] == True]
    if len(item_details) == 0:
        return (np.nan, np.nan, np.nan)

    # add time difference between each item and the sample
    item_details["time_diff"] = pd.to_datetime(date) - pd.to_datetime(
        item_details["datetime"]
    )

    # if we have sentinel-2, filter to sentinel-2 images only
    item_details["sentinel"] = item_details.platform.str.lower().str.contains(
        "sentinel"
    )
    if item_details["sentinel"].any():
        item_details = item_details[item_details["sentinel"] == True]

        # return the closest imagery by time
        listitems = item_details.sort_values(by="time_diff", ascending=True)

        for i in range(len(listitems)):
            cloud_coverage = listitems.iloc[i, 6].properties["eo:cloud_cover"]
            shadow_pixels = listitems.iloc[i, 6].properties[
                "s2:cloud_shadow_percentage"
            ]
            no_pixel_data = listitems.iloc[i, 6].properties[
                "s2:nodata_pixel_percentage"
            ]
            degraded_msi_data_percentage = listitems.iloc[i, 6].properties[
                "s2:degraded_msi_data_percentage"
            ]
            if (
                no_pixel_data < 10
                and cloud_coverage < 10
                and shadow_pixels < 10
                and degraded_msi_data_percentage < 10
            ):
                best_item = listitems.iloc[i]
    # if we have sentinel-2, filter to sentinel-2 images only
    else:
        item_details = item_details[item_details["landsat"] == True]
        listitems = item_details.sort_values(by="time_diff", ascending=True)
        for i in range(len(listitems)):
            # return the closest imagery by time
            cloud_coverage_land = listitems.iloc[i, 6].properties[
                "landsat:cloud_cover_land"
            ]
            cloud_coverage = listitems.iloc[i, 6].properties["eo:cloud_cover"]
            if cloud_coverage < 10 and cloud_coverage_land < 10:
                best_item = listitems.iloc[i]
    return (
        best_item["item_obj"],
        best_item["platform"],
        best_item["datetime"],
    )


def image_to_features(image_array):
    """
    Convert an image array of the form (color band, height, width) to a
    1-dimensional list of features. Returns a list where the first three
    values are the averages of each color band, and the second three
    values are the medians of each color band.
    """
    averages = image_array.mean(axis=(1, 2)).tolist()
    standard_dev = np.std(image_array, axis=(1, 2)).tolist()

    return averages + standard_dev

In [ ]:
BENCHMARK_DATA_DIR = DATA_DIR.parents[0] / "ticktickbloom/benchmark"

# save image arrays in case we want to generate more features
IMAGE_ARRAY_DIR = BENCHMARK_DATA_DIR / "Train"
IMAGE_ARRAY_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
IMAGE_ARRAY_DIR_TEST = BENCHMARK_DATA_DIR / "Test"
IMAGE_ARRAY_DIR_TEST.mkdir(exist_ok=True, parents=True)

In [ ]:
# take a random subset of the training data for the benchmark
train_subset = metadata[metadata["split"] == "train"].sample(n=17060, random_state=2)
test_subset = metadata[metadata["split"] == "test"]
# combine train subset with all test data
metadata_subset = pd.concat([train_subset, metadata[metadata["split"] == "test"]])
metadata_subset.split.value_counts(dropna=False)

In [ ]:
test_subset

In [ ]:
import rioxarray
import imageio
from IPython.display import Image
from PIL import Image as PILImage
from joblib import Parallel, delayed
import multiprocessing
import pandas as pd


def extract_images(row):
    pass
    # check if we've already saved the selected image array
    image_array_pth = IMAGE_ARRAY_DIR / f"{row.uid}.npy"

    if image_array_pth.exists():
        with open(image_array_pth, "rb") as f:
            image_array = np.load(f)

    # search and load the image array if not
    else:
        try:
            ## QUERY STAC API
            # get query ranges for location and date
            search_bbox = get_bounding_box(
                row.latitude, row.longitude, meter_buffer=50000
            )
            date_range = get_date_range(row.date, time_buffer_days=15)

            # search the planetary computer
            search = catalog.search(
                collections=["sentinel-2-l2a", "landsat-c2-l2"],
                bbox=search_bbox,
                datetime=date_range,
            )
            items = [item for item in search.get_all_items()]

            ## GET BEST IMAGE
            if len(items) == 0:
                pass
            else:
                best_item, item_platform, item_date = select_best_item(
                    items, row.date, row.latitude, row.longitude
                )
                # add to dictionary tracking best items
                selected_items[row.uid] = {
                    "item_object": best_item,
                    "item_platform": item_platform,
                    "item_date": item_date,
                }

            ## CONVERT TO FEATURES
            # get small bbox just for features
            feature_bbox = get_bounding_box(
                row.latitude, row.longitude, meter_buffer=3000
            )

            # crop the image
            if "sentinel" in item_platform.lower():
                image_array = crop_sentinel_image(best_item, feature_bbox)
            else:
                image_array = crop_landsat_image(best_item, feature_bbox)

            # save image array so we don't have to rerun
            image = np.transpose(image_array, axes=[1, 2, 0]).astype(np.uint8)
            # selected_items[row.uid] = {"image_array": image_array}
            imageio.imwrite(IMAGE_ARRAY_DIR / f"{row.uid}.png", image)

        # keep track of any that ran into errors without interrupting the process
        except:
            errored_ids.append(row.uid)
    return "ok"

# Import train & test images

In [ ]:
import rioxarray
import imageio
from IPython.display import Image
from PIL import Image as PILImage
from joblib import Parallel, delayed
import multiprocessing

# this cell takes a LONG time because it iterates over all data!

# save outputs in dictionaries
selected_items = {}
selected_images = {}
features_dict = {}
errored_ids = []
found_images = 0
bestitemselected = 0

for row in tqdm(train_subset.itertuples(), total=len(train_subset)):
    image_array_pth = IMAGE_ARRAY_DIR / f"{row.uid}.npy"

    if image_array_pth.exists():
        with open(image_array_pth, "rb") as f:
            image_array = np.load(f)

    # search and load the image array if not
    else:
        try:
            ## QUERY STAC API
            # get query ranges for location and date
            search_bbox = get_bounding_box(
                row.latitude, row.longitude, meter_buffer=50000
            )
            date_range = get_date_range(row.date, time_buffer_days=15)

            # search the planetary computer
            search = catalog.search(
                collections=["sentinel-2-l2a", "landsat-c2-l2"],
                bbox=search_bbox,
                datetime=date_range,
            )
            items = [item for item in search.get_all_items()]

            ## GET BEST IMAGE
            if len(items) == 0:
                pass
            else:
                best_item, item_platform, item_date = select_best_item(
                    items, row.date, row.latitude, row.longitude
                )
                # add to dictionary tracking best items
                selected_items[row.uid] = {
                    "item_object": best_item,
                    "item_platform": item_platform,
                    "item_date": item_date,
                }

            ## CONVERT TO FEATURES
            # get small bbox just for features
            feature_bbox = get_bounding_box(
                row.latitude, row.longitude, meter_buffer=3000
            )

            # crop the image
            if "sentinel" in item_platform.lower():
                image_array = crop_sentinel_image(best_item, feature_bbox)
            else:
                image_array = crop_landsat_image(best_item, feature_bbox)

            # save image array so we don't have to rerun
            image = np.transpose(image_array, axes=[1, 2, 0]).astype(np.uint8)
            # selected_items[row.uid] = {"image_array": image_array}
            imageio.imwrite(IMAGE_ARRAY_DIR / f"{row.uid}.png", image)

        # keep track of any that ran into errors without interrupting the process
        except:
            errored_ids.append(row.uid)

In [ ]:
import rioxarray
import imageio
from IPython.display import Image
from PIL import Image as PILImage
from joblib import Parallel, delayed
import multiprocessing

# this cell takes a LONG time because it iterates over all data!

# save outputs in dictionaries
selected_items_test = {}
selected_images_test = {}
features_dict_test = {}
errored_ids_test = []
found_images_test = 0
bestitemselected_test = 0

for row in tqdm(test_subset.itertuples(), total=len(test_subset)):
    image_array_pth = IMAGE_ARRAY_DIR_TEST / f"{row.uid}.npy"  ##????????

    if image_array_pth.exists():
        with open(image_array_pth, "rb") as f:
            image_array = np.load(f)

    # search and load the image array if not
    else:
        try:
            ## QUERY STAC API
            # get query ranges for location and date
            search_bbox = get_bounding_box(
                row.latitude, row.longitude, meter_buffer=50000
            )
            date_range = get_date_range(row.date, time_buffer_days=15)

            # search the planetary computer
            search = catalog.search(
                collections=["sentinel-2-l2a", "landsat-c2-l2"],
                bbox=search_bbox,
                datetime=date_range,
            )
            items = [item for item in search.get_all_items()]

            ## GET BEST IMAGE
            if len(items) == 0:
                pass
            else:
                best_item, item_platform, item_date = select_best_item(
                    items, row.date, row.latitude, row.longitude
                )
                # add to dictionary tracking best items
                selected_items[row.uid] = {
                    "item_object": best_item,
                    "item_platform": item_platform,
                    "item_date": item_date,
                }

            ## CONVERT TO FEATURES
            # get small bbox just for features
            feature_bbox = get_bounding_box(
                row.latitude, row.longitude, meter_buffer=3000
            )

            # crop the image
            if "sentinel" in item_platform.lower():
                image_array = crop_sentinel_image(best_item, feature_bbox)
            else:
                image_array = crop_landsat_image(best_item, feature_bbox)

            # save image array so we don't have to rerun
            image = np.transpose(image_array, axes=[1, 2, 0]).astype(np.uint8)
            # selected_items[row.uid] = {"image_array": image_array}
            imageio.imwrite(IMAGE_ARRAY_DIR_TEST / f"{row.uid}.png", image)

        # keep track of any that ran into errors without interrupting the process
        except:
            errored_ids.append(row.uid)

In [ ]:
import os
import stat
from PIL import Image

paths = sorted(list(IMAGE_ARRAY_DIR.glob("*.png")))
path_list = os.path.split(paths[0])
id = path_list[1][:4]

im = Image.open(path_list[0] + "\\" + path_list[1])
np.asarray(im).shape

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pathlib import Path
import os
import stat
from PIL import Image

# 1st approach

### creating features, spectral data

4D tensors will be used as input; each channel have the information of a choosen spectral index.

In [ ]:


for row in tqdm(train_subset.itertuples(), total=len(train_subset)):
    ## QUERY STAC API
    # get query ranges for location and date
    search_bbox = get_bounding_box(row.latitude, row.longitude, meter_buffer=50000)
    date_range = get_date_range(row.date, time_buffer_days=60)

    # search the planetary computer
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=search_bbox,
        datetime=date_range,
    )
    items = [item for item in search.get_all_items()]

    ## GET BEST IMAGE
    if len(items) == 0:
        pass
    else:
        try:
            best_item, item_platform, item_date = select_best_item(
                items, row.date, row.latitude, row.longitude
            )
        except:
            pass
        # add to dictionary tracking best items
    feature_bbox = get_bounding_box(row.latitude, row.longitude, meter_buffer=3000)

    (minx, miny, maxx, maxy) = feature_bbox

    try:
        # spectral bands
        nir = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B08"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        red = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B04"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        green = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B03"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        blue = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B02"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        n_nir = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B8A"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        red_v = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B05"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()

        # different shapes
        red_v = red_v.astype(np.float32)
        red_v = transforms.Resize((605, 604))(
            transforms.ToPILImage()(red_v.transpose())
        )
        red_v = np.asarray(red_v)

        n_nir = n_nir.astype(np.float32)
        n_nir = transforms.Resize((605, 604))(
            transforms.ToPILImage()(n_nir.transpose())
        )
        n_nir = np.asarray(n_nir)

        red = red.astype(np.float32)
        red = transforms.Resize((605, 604))(transforms.ToPILImage()(red.transpose()))
        red = np.asarray(red)

        blue = blue.astype(np.float32)
        blue = transforms.Resize((605, 604))(transforms.ToPILImage()(blue.transpose()))
        blue = np.asarray(blue)

        green = green.astype(np.float32)
        green = transforms.Resize((605, 604))(
            transforms.ToPILImage()(green.transpose())
        )
        green = np.asarray(green)

        nir = n_nir.astype(np.float32)
        nir = transforms.Resize((605, 604))(transforms.ToPILImage()(nir.transpose()))
        nir = np.asarray(nir)

        # spectral indices
        NDVI = (nir - red) / (nir + red)
        NDCI = (red_v - red) / (red_v + red)
        B8AB4 = (n_nir - red) / (n_nir + red)
        B3B2 = (green - blue) / (green + blue)
        im = np.stack((NDVI, NDCI, B8AB4, B3B2), axis=-1)
        path = "Train_tensors/" + row.uid + ".npy"
        np.save(DATA_DIR / path, im.T)
    except:
        pass

In [ ]:
for row in tqdm(test_subset.itertuples(), total=len(test_subset)):
    ## QUERY STAC API
    # get query ranges for location and date
    search_bbox = get_bounding_box(row.latitude, row.longitude, meter_buffer=50000)
    date_range = get_date_range(row.date, time_buffer_days=15)

    # search the planetary computer
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=search_bbox,
        datetime=date_range,
    )
    items = [item for item in search.get_all_items()]

    ## GET BEST IMAGE
    if len(items) == 0:
        pass
    else:
        try:
            best_item, item_platform, item_date = select_best_item(
                items, row.date, row.latitude, row.longitude
            )
        except:
            pass
        # add to dictionary tracking best items
    feature_bbox = get_bounding_box(row.latitude, row.longitude, meter_buffer=3000)

    (minx, miny, maxx, maxy) = feature_bbox

    try:
        # spectral bands
        nir = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B08"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        red = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B04"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        green = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B03"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        blue = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B02"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        n_nir = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B8A"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()
        red_v = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B05"].href)).rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
        ).to_numpy()

        # different shapes
        red_v = red_v.astype(np.float32)
        red_v = transforms.Resize((605, 604))(
            transforms.ToPILImage()(red_v.transpose())
        )
        red_v = np.asarray(red_v)

        n_nir = n_nir.astype(np.float32)
        n_nir = transforms.Resize((605, 604))(
            transforms.ToPILImage()(n_nir.transpose())
        )
        n_nir = np.asarray(n_nir)

        red = red.astype(np.float32)
        red = transforms.Resize((605, 604))(transforms.ToPILImage()(red.transpose()))
        red = np.asarray(red)

        blue = blue.astype(np.float32)
        blue = transforms.Resize((605, 604))(transforms.ToPILImage()(blue.transpose()))
        blue = np.asarray(blue)

        green = green.astype(np.float32)
        green = transforms.Resize((605, 604))(
            transforms.ToPILImage()(green.transpose())
        )
        green = np.asarray(green)

        nir = n_nir.astype(np.float32)
        nir = transforms.Resize((605, 604))(transforms.ToPILImage()(nir.transpose()))
        nir = np.asarray(nir)

        # spectral indices
        NDVI = (nir - red) / (nir + red)
        NDCI = (red_v - red) / (red_v + red)
        B8AB4 = (n_nir - red) / (n_nir + red)
        B3B2 = (green - blue) / (green + blue)
        im = np.stack((NDVI, NDCI, B8AB4, B3B2), axis=-1)
        path = "Test_tensors/" + row.uid + ".npy"
        np.save(DATA_DIR / path, im.T)
    except:
        pass

### Dataloaders

In [ ]:
import random

dir = DATA_DIR / "Train_tensors"
paths = sorted(list(dir.glob("*.npy")))
test_size = int(0.2 * len(paths))
val_paths = random.sample(paths, test_size)
train_paths = [x for x in paths if x not in val_paths]

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, paths):
        self.tensor_paths = paths

    def load_tensor(self, index: int):
        tensor_path = self.tensor_paths[index]
        return np.load(tensor_path)

    def __len__(self):
        return len(self.tensor_paths)

    def __getitem__(self, index):
        img = self.load_tensor(index)
        tensor_path = self.tensor_paths[index]
        path_list = os.path.split(tensor_path)
        id = path_list[1][:4]
        y = train_labels[train_labels.uid == id]["severity"][
            train_labels[train_labels.uid == id].index[0]
        ]
        return img, y - 1

In [ ]:
train_dataset = Dataset(train_paths)
val_dataset = Dataset(val_paths)

In [ ]:
train_dataloader = DataLoader(
    dataset=train_dataset, batch_size=4, num_workers=0, shuffle=True
)

test_dataloader = DataLoader(
    dataset=val_dataset, batch_size=4, num_workers=0, shuffle=True
)

### Model

Resnet50 doesn't work with 4 channels :: to adress thsi issue we can add a conv layer before resnet

Custumized model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()

        # Convolutional layer to reduce number of channels from 4 to 3
        self.conv = nn.Conv2d(
            in_channels=4, out_channels=3, kernel_size=3, stride=1, padding=1
        )

        # Load a pre-trained ResNet50 model
        self.resnet = models.resnet50(pretrained=False)

    def forward(self, x):
        # Pass the input through the convolutional layer
        x = self.conv(x)

        # replace the last fully-connected layer with our own classifier
        num_ftrs = self.resnet.fc.in_features
        num_classes = 5
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

        # Pass the output through the ResNet50 model
        x = self.resnet(x)

        return x

In [ ]:
model = CustomModel()
# summary(model, (4, 605, 604))

### Train

In [ ]:
def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
):
    model.train()

    train_loss = 0

    for batch, (image, label) in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(image)

        loss = loss_fn(output, label)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()


def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
):
    model.eval()

    test_loss = 0
    with torch.no_grad():
        for i, (image, label) in enumerate(dataloader):
            output = model(image)
            loss = loss_fn(output, label)
            test_loss += loss.item()
    test_loss /= len(dataloader)
    return test_loss


def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs,
):
    # Create empty results dictionary
    results = {"train_loss": [], "test_loss": []}

    for epoch in tqdm(range(epochs)):
        train_loss = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
        )

        test_loss = test_step(model=model, dataloader=test_dataloader, loss_fn=loss_fn)

        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"test_loss: {test_loss:.4f} | "
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)

    # Return the filled results at the end of the epochs
    return results

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model_result = train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=train_dataloader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=50,
)

Not enough RAM!!!

# 2nd approach

### Creating Tabular data

In [ ]:
#this algorith will be used to extract water and algae pixels
def otsu_binarization(image, mask=None):
    if mask is None:
        mask = np.full_like(image, True, dtype=bool)

    pixel_values, counts = np.unique(image[mask], return_counts=True)
    probabilities = counts / image[mask].size
    thresholds = pixel_values[1:]

    best_threshold = None
    max_y = float("-inf")
    for index, threshold in tqdm(enumerate(thresholds, 1), total=thresholds.shape[0]):
        pixel_value0 = pixel_values[pixel_values < threshold]
        pixel_value1 = pixel_values[pixel_values >= threshold]

        probabilities0 = probabilities[pixel_values < threshold]
        probabilities1 = probabilities[pixel_values >= threshold]
        w0 = probabilities0.sum()
        w1 = probabilities1.sum()
        mu0 = (pixel_value0 * probabilities0).sum() / w0
        mu1 = (pixel_value1 * probabilities1).sum() / w1

        u = w0 * mu0 + w1 * mu1
        y = w0 * (mu0 - u) ** 2 + w1 * (mu1 - u) ** 2

        if y > max_y:
            best_threshold = threshold
            max_y = y

    return np.all(np.array([image >= best_threshold, mask]), axis=0)

In [ ]:
#resizing images
def resize_images(*imgs):
    img_shapes = [img.shape for img in imgs]
    h, w = float("inf"), float("inf")
    for shape in img_shapes:
        h_temp, w_temp = shape
        if h_temp < h:
            h = h_temp
        if w_temp < w:
            w = w_temp
    return (
        cv2.resize(img, dsize=(h, w), interpolation=cv2.INTER_CUBIC) for img in imgs
    )

### creating features; mean & std of selected spectral indices

In [ ]:
features_dict = {}
ids = []
import os
import stat
from PIL import Image

dir = DATA_DIR / "Train_tensors"
paths = sorted(list(dir.glob("*.npy")))


for row in tqdm(train_subset[945:].itertuples(), total=len(train_subset[945:])):
    ## QUERY STAC API
    # get query ranges for location and date
    search_bbox = get_bounding_box(row.latitude, row.longitude, meter_buffer=50000)
    date_range = get_date_range(row.date, time_buffer_days=60)

    # search the planetary computer
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=search_bbox,
        datetime=date_range,
    )
    items = [item for item in search.get_all_items()]

    ## GET BEST IMAGE
    if len(items) == 0:
        pass
    else:
        try:
            best_item, item_platform, item_date = select_best_item(
                items, row.date, row.latitude, row.longitude
            )
        except:
            pass
        # add to dictionary tracking best items
    feature_bbox = get_bounding_box(row.latitude, row.longitude, meter_buffer=3000)

    (minx, miny, maxx, maxy) = feature_bbox

    try:
        # spectral bands
        nir = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B08"].href))
            .rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
            .to_numpy()[0]
        )
        red = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B04"].href))
            .rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
            .to_numpy()[0]
        )
        green = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B03"].href))
            .rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
            .to_numpy()[0]
        )
        blue = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B02"].href))
            .rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
            .to_numpy()[0]
        )
        n_nir = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B8A"].href))
            .rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
            .to_numpy()[0]
        )
        red_v = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B05"].href))
            .rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
            .to_numpy()[0]
        )

        b11 = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B11"].href))
            .rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
            .to_numpy()[0]
        )

        b06 = (
            rioxarray.open_rasterio(pc.sign(best_item.assets["B06"].href))
            .rio.clip_box(
                minx=minx,
                miny=miny,
                maxx=maxx,
                maxy=maxy,
                crs="EPSG:4326",
            )
            .to_numpy()[0]
        )

        # resizing
        b11, blue, n_nir, red_v, b06, red, nir, green = resize_images(
            b11, blue, n_nir, red_v, b06, red, nir, green
        )

        # spectral indices
        NDVI = (nir - red) / (nir + red)
        NDCI = (red_v - red) / (red_v + red)
        B8AB4 = (n_nir - red) / (n_nir + red)
        B3B2 = (green - blue) / (green + blue)
        swi = (red_v - b11) / (red_v + b11)
        abdi = (b06 - red - (nir - red) * (740.5 - 15) / (832.8 - 15)) - (
            red - 0.5 * green
        )

        # water & algae extraction
        extracted_water = otsu_binarization(swi)
        extracted_algae = otsu_binarization(abdi, extracted_water)

        f1, f2 = [np.mean(NDVI)], [np.std(NDVI)]
        f3, f4 = [np.mean(NDCI)], [np.std(NDCI)]
        f5, f6 = [np.mean(B8AB4)], [np.std(B8AB4)]
        f7, f8 = [np.mean(B3B2)], [np.std(B3B2)]
        f9, f10 = [np.mean(extracted_algae)], [np.std(extracted_algae)]
        features = f1 + f2 + f3 + f4 + f5 + f6 + f7 + f8 + f9 + f10

        features_dict[row.uid] = [row.uid] + features
    except:
        pass

### Storing data in a csv file

In [ ]:
import csv

# write to csv file
with open('data.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=features_dict.keys())
    writer.writeheader()
    writer.writerows([dict(zip(features_dict.keys(), t)) for t in zip(*features_dict.values())])


### dataframe

In [ ]:
# bring features into a dataframe
train_set = pd.DataFrame(features_dict).T
train_set.columns = [
    "uid",
    "NDVI_average",
    "NDCI_average",
    "B8AB4_average",
    "B3B2_average",
    "algae_average",
    "NDVI_std",
    "NDCI_std",
    "B8AB4_std",
    "B3B2_std",
    "algae_std"
]
train_set

In [ ]:
dataframe = train_set.merge(
    train_labels, how="left", left_on="uid", right_on="uid", validate="1:1"
)

In [ ]:
dataframe

### Machine learning Models

In [ ]:
X = dataframe.drop(["density", "severity"], axis=1, inplace=False)
y = dataframe["severity"]

In [ ]:
y

In [ ]:
X.info()

In [ ]:
X.fillna(X.mean(), inplace=True)

In [ ]:
X.info()

In [ ]:
dummy_reg = pd.get_dummies(X["region"])
df = pd.concat([X, dummy_reg], axis=1)
X = df.drop(["uid"], axis=1, inplace=False)
X

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_test_northeast = X[X["region"] == "northeast"]
X_test_midwest = X[X["region"] == "midwest"]
X_test_west = X[X["region"] == "west"]
X_test_south = X[X["region"] == "south"]

y_test_northeast = dataframe[dataframe["region"] == "northeast"]["severity"]
y_test_midwest = dataframe[dataframe["region"] == "midwest"]["severity"]
y_test_west = dataframe[dataframe["region"] == "west"]["severity"]
y_test_south = dataframe[dataframe["region"] == "south"]["severity"]

clf = DecisionTreeClassifier()
X_train1 = X_train.drop(["region"], axis=1, inplace=False)
clf.fit(X_train1, y_train)

X_test.drop("region", axis=1, inplace=True)
X_test_northeast.drop("region", axis=1, inplace=True)
X_test_midwest.drop("region", axis=1, inplace=True)
X_test_west.drop("region", axis=1, inplace=True)
X_test_south.drop("region", axis=1, inplace=True)

y_pred = clf.predict(X_test)
y_pred_northeast = clf.predict(X_test_northeast)
y_pred_midwest = clf.predict(X_test_midwest)
y_pred_west = clf.predict(X_test_west)
y_pred_south = clf.predict(X_test_south)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# avg_region_rmse
avg_reg_rmse = (
    1
    / 4
    * (
        np.sqrt(mean_squared_error(y_test_northeast, y_pred_northeast))
        + np.sqrt(mean_squared_error(y_test_midwest, y_pred_midwest))
        + np.sqrt(mean_squared_error(y_test_west, y_pred_west))
        + np.sqrt(mean_squared_error(y_test_south, y_pred_south))
    )
)
print("avg_region_rmse:", avg_reg_rmse)
print("rmse_norwest:", np.sqrt(mean_squared_error(y_test_northeast, y_pred_northeast)))
print("rmse_midwest:", np.sqrt(mean_squared_error(y_test_midwest, y_pred_midwest)))
print("rmse_south:", np.sqrt(mean_squared_error(y_test_south, y_pred_south)))
print("rmse_north:", np.sqrt(mean_squared_error(y_test_west, y_pred_west)))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_test_northeast = X[X["region"] == "northeast"]
X_test_midwest = X[X["region"] == "midwest"]
X_test_west = X[X["region"] == "west"]
X_test_south = X[X["region"] == "south"]

y_test_northeast = dataframe[dataframe["region"] == "northeast"]["severity"]
y_test_midwest = dataframe[dataframe["region"] == "midwest"]["severity"]
y_test_west = dataframe[dataframe["region"] == "west"]["severity"]
y_test_south = dataframe[dataframe["region"] == "south"]["severity"]

clf = RandomForestClassifier()
X_train1 = X_train.drop(["region"], axis=1, inplace=False)
clf.fit(X_train1, y_train)

X_test.drop("region", axis=1, inplace=True)
X_test_northeast.drop("region", axis=1, inplace=True)
X_test_midwest.drop("region", axis=1, inplace=True)
X_test_west.drop("region", axis=1, inplace=True)
X_test_south.drop("region", axis=1, inplace=True)

y_pred = clf.predict(X_test)
y_pred_northeast = clf.predict(X_test_northeast)
y_pred_midwest = clf.predict(X_test_midwest)
y_pred_west = clf.predict(X_test_west)
y_pred_south = clf.predict(X_test_south)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# avg_region_rmse
avg_reg_rmse = (
    1
    / 4
    * (
        np.sqrt(mean_squared_error(y_test_northeast, y_pred_northeast))
        + np.sqrt(mean_squared_error(y_test_midwest, y_pred_midwest))
        + np.sqrt(mean_squared_error(y_test_west, y_pred_west))
        + np.sqrt(mean_squared_error(y_test_south, y_pred_south))
    )
)
print("avg_region_rmse:", avg_reg_rmse)
print("rmse_norwest:", np.sqrt(mean_squared_error(y_test_northeast, y_pred_northeast)))
print("rmse_midwest:", np.sqrt(mean_squared_error(y_test_midwest, y_pred_midwest)))
print("rmse_south:", np.sqrt(mean_squared_error(y_test_south, y_pred_south)))
print("rmse_west:", np.sqrt(mean_squared_error(y_test_west, y_pred_west)))